In [1]:
import numpy as np
import pandas as pd
# library for splitting training-testing
from sklearn.model_selection import train_test_split
# library for classification
from sklearn.metrics import accuracy_score

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler
from sklearn import pipeline

In [17]:
x_train = pd.read_feather("Data/x_train__IterativeImputer_genFeats.ftr")
x_test = pd.read_feather("Data/x_test__IterativeImputer_genFeats.ftr")

x_tr = pd.read_csv("Preds/preds_tr.csv")
x_val_1 = pd.read_csv("Preds/preds_val.csv")
x_test_1 = pd.read_csv("Preds/preds_test.csv")

y_train = pd.read_csv("Data/train_y.csv")
y_test = pd.read_csv("Data/Sample_Output.csv", names = ['ID', 'Target'], header=None)

# Test Train Split

In [18]:
# scaler = StandardScaler().fit(x_train)
# x_train = scaler.transform(x_train)

In [19]:
x_train_1, x_val, y_train_1, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=59, stratify=y_train)

In [30]:
x_train_1 = pd.concat([x_train_1.reset_index(drop=True), x_tr], axis=1)
x_val = pd.concat([x_val.reset_index(drop=True), x_val_1], axis=1)
x_test = pd.concat([x_test.reset_index(drop=True), x_test_1], axis=1)

In [7]:
split_ind = [-1]*(len(x_train))
for ind in list(x_val.index.values):
    split_ind[ind] = 0

In [8]:
ps = PredefinedSplit(split_ind)

In [31]:
model = LogisticRegression(multi_class='multinomial',
                          random_state=59, n_jobs=-1,verbose=True,
                          warm_start=True,C=1, max_iter=1000, penalty='none')

In [32]:
scl = StandardScaler()

pipe = pipeline.Pipeline(steps=[('scl', scl), ('model', model)], verbose=True)

In [33]:
%%time

pipe.fit(x_train_1, y_train_1.values.reshape(-1))

[Pipeline] ............... (step 1 of 2) Processing scl, total=  45.4s


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         1644     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.40760D+05    |proj g|=  1.91553D+05


 This problem is unconstrained.



At iterate   50    f=  1.81393D+05    |proj g|=  2.80697D+02

At iterate  100    f=  1.81147D+05    |proj g|=  1.19271D+02

At iterate  150    f=  1.81081D+05    |proj g|=  4.85410D+01

At iterate  200    f=  1.81059D+05    |proj g|=  3.80194D+01

At iterate  250    f=  1.81045D+05    |proj g|=  2.98509D+01

At iterate  300    f=  1.81034D+05    |proj g|=  3.76866D+01

At iterate  350    f=  1.81020D+05    |proj g|=  8.09330D+01

At iterate  400    f=  1.81012D+05    |proj g|=  2.77495D+01

At iterate  450    f=  1.81008D+05    |proj g|=  2.94495D+01

At iterate  500    f=  1.81004D+05    |proj g|=  3.88952D+01

At iterate  550    f=  1.81001D+05    |proj g|=  2.05569D+01

At iterate  600    f=  1.80999D+05    |proj g|=  1.84383D+01

At iterate  650    f=  1.80997D+05    |proj g|=  3.24309D+01

At iterate  700    f=  1.80994D+05    |proj g|=  1.81932D+01

At iterate  750    f=  1.80992D+05    |proj g|=  1.76467D+01

At iterate  800    f=  1.80991D+05    |proj g|=  1.34959D+01

At iter

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 33.1min finished


Pipeline(steps=[('scl', StandardScaler()),
                ('model',
                 LogisticRegression(C=1, max_iter=1000,
                                    multi_class='multinomial', n_jobs=-1,
                                    penalty='none', random_state=59,
                                    verbose=True, warm_start=True))],
         verbose=True)

In [34]:
preds_tr = pipe.predict(x_train_1,)
preds_val = pipe.predict(x_val)

In [35]:
accuracy_score(y_train_1, preds_tr)

0.8135465909207437

In [36]:
accuracy_score(y_val, preds_val)

0.814881531734387

In [37]:
probs_tr = pipe.predict_proba(x_train_1)
probs_val = pipe.predict_proba(x_val)

In [38]:
np.save("Preds/LR_val_stack", probs_val)
np.save("Preds/LR_tr_stack", probs_tr)

probs_test = pipe.predict_proba(x_test)
np.save("Preds/LR_test_stack", probs_test)